In [52]:
import pandas as pd
import numpy as np
import os

patients_dfs = []


for item in os.listdir(r"C:\Users\yasha\PharmaHacks2019\cleaned_data"):
    df1 = pd.read_csv(os.path.join(r"C:\Users\yasha\PharmaHacks2019\cleaned_data", item), sep = '\t', skiprows = 0, header=0, index_col=0)
    patients_dfs.append(df1)

patients_dfs[0]

,DR Speciality,Payment Method #5,Payment Method #4,DR PROVINCE,Dosage,DR ID,Payment Method #1,Status Group,Status Description,Frequency,...,Patient Receiving Free Drug,Case State,Age Range,Day Enrollment Received,Day Enrollment Completed,State Change Day,On Drug Start Day,Last On Drug Day,Re-Engagement Day,Re-Engagement On Drug Start Day
Patient ID,,,,,,,,,,,,,,,,,,,,,
Patient_165,0.0,0.0,0.0,0.0,0.0,385.0,1.0,0.0,4.0,0.0,...,0.0,0.0,50,42198,42204,42374,0,0,0,0
Patient_166,0.0,0.0,0.0,4.0,3.0,396.0,0.0,0.0,18.0,2.0,...,0.0,0.0,60,42200,42217,42219,0,0,0,0
Patient_167,0.0,0.0,0.0,5.0,0.0,407.0,1.0,1.0,20.0,0.0,...,0.0,1.0,30,42203,42213,42203,42254,0,0,0
Patient_168,0.0,0.0,0.0,5.0,0.0,418.0,1.0,0.0,13.0,0.0,...,0.0,0.0,50,42204,42213,42749,42277,42675,0,0
Patient_169,0.0,0.0,0.0,4.0,2.0,429.0,1.0,0.0,0.0,0.0,...,0.0,0.0,60,42204,42214,42925,42238,42878,0,0
Patient_170,0.0,0.0,0.0,5.0,0.0,440.0,0.0,1.0,20.0,0.0,...,0.0,1.0,70,42211,42213,42211,42288,0,0,0
Patient_171,0.0,0.0,0.0,5.0,0.0,451.0,0.0,1.0,20.0,0.0,...,0.0,1.0,50,42213,42213,42213,42245,0,0,0
Patient_172,0.0,0.0,0.0,5.0,0.0,462.0,1.0,1.0,20.0,0.0,...,0.0,1.0,40,42213,42217,42213,42260,0,0,0
Patient_173,0.0,0.0,0.0,5.0,0.0,473.0,1.0,1.0,20.0,0.0,...,0.0,1.0,40,42213,42214,42213,42272,0,0,0


In [53]:
# Cleaning the data sets: 
def correct_shift(x):
    a = x["Last On Drug Day"] 
    b = x["On Drug Start Day"]
    var = a-b if a != 0 else b
    return var

# 1. Removing patients with On Drug Start Day, Last On Drug Day = 0 (i.e never were in program at all...)
for index, df in enumerate(patients_dfs):
    df["Enrollment Time"] = df["Day Enrollment Completed"] - df["Day Enrollment Received"]
    df["On Drug Time"] = df[["Last On Drug Day", "On Drug Start Day"]].apply(correct_shift, axis=1)
    df["# of Payment Methods"] = df[["Payment Method #4", "Payment Method #5", "Payment Method #1", "Payment Method #3", "Payment Method #2"]].sum(axis=1)
    
    df = df.drop(columns = ["Day Enrollment Completed", "Day Enrollment Received", "Last On Drug Day", "On Drug Start Day",
                           "Payment Method #4", "Payment Method #5", "Payment Method #1", "Payment Method #3", "Payment Method #2"])
    
    patients_dfs[index] = df

In [54]:
patients_dfs[0]

,DR Speciality,DR PROVINCE,Dosage,DR ID,Status Group,Status Description,Frequency,Diagnosis ID,Biologic Line of Therapy,Status,Gender ID,Patient Receiving Free Drug,Case State,Age Range,State Change Day,Re-Engagement Day,Re-Engagement On Drug Start Day,Enrollment Time,On Drug Time,# of Payment Methods
Patient ID,,,,,,,,,,,,,,,,,,,,
Patient_165,0.0,0.0,0.0,385.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50,42374,0,0,6,0,1.0
Patient_166,0.0,4.0,3.0,396.0,0.0,18.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,60,42219,0,0,17,0,1.0
Patient_167,0.0,5.0,0.0,407.0,1.0,20.0,0.0,0.0,0.0,9.0,0.0,0.0,1.0,30,42203,0,0,10,42254,1.0
Patient_168,0.0,5.0,0.0,418.0,0.0,13.0,0.0,0.0,3.0,10.0,1.0,0.0,0.0,50,42749,0,0,9,398,1.0
Patient_169,0.0,4.0,2.0,429.0,0.0,0.0,0.0,0.0,1.0,11.0,1.0,0.0,0.0,60,42925,0,0,10,640,1.0
Patient_170,0.0,5.0,0.0,440.0,1.0,20.0,0.0,0.0,3.0,9.0,1.0,0.0,1.0,70,42211,0,0,2,42288,1.0
Patient_171,0.0,5.0,0.0,451.0,1.0,20.0,0.0,0.0,1.0,9.0,0.0,0.0,1.0,50,42213,0,0,0,42245,1.0
Patient_172,0.0,5.0,0.0,462.0,1.0,20.0,0.0,0.0,1.0,9.0,0.0,0.0,1.0,40,42213,0,0,4,42260,1.0
Patient_173,0.0,5.0,0.0,473.0,1.0,20.0,0.0,0.0,3.0,9.0,1.0,0.0,1.0,40,42213,0,0,1,42272,1.0


In [36]:
# 2. Data Exploration, Correlation, Chi-Squared
from scipy import stats
base_df = patients_dfs[len(patients_dfs)-1]
feat_num = base_df.shape[1]
features = base_df.columns
pvals = np.zeros((feat_num,feat_num))

for i, feature_i in enumerate(features):
    for j, feature_j in enumerate(features):
        chi2, p, dof, expec = stats.chi2_contingency((pd.crosstab(base_df[feature_i],base_df[feature_j])))
        pvals[i][j] = p

In [44]:
# P-value matrix

pvals_df = pd.DataFrame(pvals, columns=features)
pvals_df.index = features

,Status,Case State,Biologic Line of Therapy,Frequency,DR Speciality,Diagnosis ID,Patient Receiving Free Drug,DR PROVINCE,DR ID,Dosage,Status Description,Gender ID,Status Group,Age Range,State Change Day,Re-Engagement Day,Re-Engagement On Drug Start Day,Enrollment Time,On Drug Time,# of Payment Methods
Status,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.130953e-118,0.000000e+00,4.172385e-213,2.706324e-50,5.130695e-148,0.000000e+00,0.000000e+00,1.268266e-129,0.000000e+00,7.192860e-10,1.777402e-51,1.000000e+00,1.000000e+00,0.000000e+00,5.931987e-38,6.517338e-10
Case State,0.000000e+00,0.000000e+00,1.713408e-84,1.302752e-34,4.200337e-08,4.396713e-30,1.018326e-191,1.577536e-05,2.486094e-45,1.691368e-30,0.000000e+00,6.436989e-07,0.000000e+00,2.148062e-02,3.001135e-14,3.319653e-01,2.836562e-01,5.937269e-21,0.000000e+00,8.015049e-12
Biologic Line of Therapy,0.000000e+00,1.713408e-84,0.000000e+00,3.084422e-109,7.838629e-182,2.479637e-281,6.673174e-10,8.927353e-31,1.142072e-114,7.416816e-113,0.000000e+00,3.313150e-44,0.000000e+00,4.441275e-20,5.216033e-07,1.000000e+00,1.000000e+00,0.000000e+00,5.439822e-24,8.422842e-05
Frequency,0.000000e+00,1.302752e-34,3.084422e-109,0.000000e+00,4.002044e-05,0.000000e+00,4.712812e-75,5.911598e-42,0.000000e+00,0.000000e+00,1.706347e-305,5.878482e-26,1.279572e-81,2.194487e-03,7.975624e-119,1.000000e+00,1.000000e+00,0.000000e+00,2.624103e-31,7.246301e-04
DR Speciality,5.130953e-118,4.200337e-08,7.838629e-182,4.002044e-05,0.000000e+00,0.000000e+00,3.007653e-01,0.000000e+00,0.000000e+00,0.000000e+00,3.295235e-50,1.412542e-27,7.661323e-21,1.985365e-31,9.990970e-01,1.000000e+00,1.000000e+00,1.000000e+00,6.054618e-18,6.454059e-01
Diagnosis ID,0.000000e+00,4.396713e-30,2.479637e-281,0.000000e+00,0.000000e+00,0.000000e+00,9.738110e-12,2.994945e-180,0.000000e+00,0.000000e+00,0.000000e+00,2.465808e-85,3.808597e-96,3.879538e-178,2.840205e-19,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,2.845251e-01
Patient Receiving Free Drug,4.172385e-213,1.018326e-191,6.673174e-10,4.712812e-75,3.007653e-01,9.738110e-12,0.000000e+00,1.240308e-24,1.229458e-17,1.184800e-30,4.192789e-214,3.050679e-01,9.913045e-220,6.917934e-08,4.735067e-02,3.350557e-01,1.506723e-01,1.000000e+00,3.777704e-27,3.308217e-20
DR PROVINCE,2.706324e-50,1.577536e-05,8.927353e-31,5.911598e-42,0.000000e+00,2.994945e-180,1.240308e-24,0.000000e+00,0.000000e+00,1.991193e-54,3.466468e-52,4.317609e-05,1.783519e-07,4.864173e-08,6.431270e-01,1.196757e-197,0.000000e+00,0.000000e+00,2.003212e-01,1.604322e-06
DR ID,5.130695e-148,2.486094e-45,1.142072e-114,0.000000e+00,0.000000e+00,0.000000e+00,1.229458e-17,0.000000e+00,0.000000e+00,0.000000e+00,2.474531e-83,3.990725e-43,9.910820e-96,2.610969e-32,4.423332e-01,1.000000e+00,1.000000e+00,1.138209e-315,1.000000e+00,2.745301e-01
Dosage,0.000000e+00,1.691368e-30,7.416816e-113,0.000000e+00,0.000000e+00,0.000000e+00,1.184800e-30,1.991193e-54,0.000000e+00,0.000000e+00,0.000000e+00,3.568190e-27,1.533833e-74,4.469843e-22,7.766460e-115,9.999993e-01,9.999999e-01,0.000000e+00,7.168929e-24,4.515786e-04


In [64]:
#get_deltas_df(list=["names of the df's we want"])
from scipy import stats

def chi_square(df,ls_categ):
    df_new = get_deltas_df(df)
    base_df = df_new[ls_categ] # assuming ls_categ contains list of categorical variables
    
    feat_num = base_df.shape[1]
    features = base_df.columns
    pvals = np.zeros((feat_num,feat_num))

    for i, feature_i in enumerate(features):
        for j, feature_j in enumerate(features):
            chi2, p, dof, expec = stats.chi2_contingency((pd.crosstab(base_df[feature_i],base_df[feature_j])))
            pvals[i][j] = p
    return pvals

def correlation(df,ls_contin):
    df_new = get_deltas_df(df)
    base_df = df_new[ls_continuous] # assuming ls_categ contains list of categorical variables
    
    feat_num = base_df.shape[1]
    features = base_df.columns
    CorrlMatrx=np.corrcoef(df.values, rowvar=False)
    return CorrlMatrix
    
def find_best_features(X, y):
    '''
        X - numpy array 
        y - numpy array of response variable
        
    '''
    import pandas as pd
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.feature_selection import SelectFromModel
    
    # Selector obj using Random Forest classifier
    rf = RandomForestClassifier(n_estimators=1000, max_depth=5, random_state=0)
    sfm = SelectFromModel(rf, threshold=0.15)
    
    # Train Selector obj
    sfm.fit(X, y)
    
    # Output Data Subset w/ Important Features
    return sfm.transform(X)

In [ ]:
# 3. Use following functions to make future predictions of deltas; applying feature selection

get_deltas_df()

